In [2]:
import os
import torch
import numpy as np
import nibabel as nib
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoModel
import torch.nn as nn
import pandas as pd
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import umap
import matplotlib.pyplot as plt


/scratch/users/maxvpuyv/anaconda3/envs/brain/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# --- SETTINGS ---
DINO_MODEL_NAME = "facebook/dinov2-base"
CHECKPOINT_PATH = "/oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/runs/dino_bbox_4batch/checkpoints/checkpoint_final.pt"
PDGM_DIR = "/oak/stanford/groups/ogevaert/data/brain_mri_tumor_project/UCSF-PDGM-v3"
METADATA_CSV = "/oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/data/metadata/PGDM/UCSF-PDGM-metadata_v2.csv"
SAVE_PATH = "/oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/data/features/dino_pdgm_bbox_features_1e4lr150000.npz"
CROP_SIZE = (80, 96)  # (height, width)
MARGIN = [10, 10]     # (height, width)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Projection Head definition ---
class ProjectionHead(nn.Module):
    def __init__(self, in_dim, out_dim=256):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.GELU(),
            nn.Linear(out_dim, out_dim)
        )
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.proj(x)

# --- Load DINO backbone and projection head ---
processor = AutoImageProcessor.from_pretrained(DINO_MODEL_NAME)
vit = AutoModel.from_pretrained(DINO_MODEL_NAME).to(DEVICE)
proj_head = ProjectionHead(vit.config.hidden_size).to(DEVICE)

ckpt = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
vit.load_state_dict(ckpt["backbone"])
proj_head.load_state_dict(ckpt["head"])
vit.eval()
proj_head.eval()

# --- Helper: Crop to tumor bbox + margin for 2D slice ---
def crop_slice_to_bbox_with_margin(img_slice, mask_slice, margin, crop_size):
    if np.sum(mask_slice) == 0:
        # No tumor, center crop
        h, w = img_slice.shape
        ch, cw = crop_size
        y0 = max(0, (h - ch) // 2)
        x0 = max(0, (w - cw) // 2)
        return img_slice[y0:y0+ch, x0:x0+cw]
    coords = np.array(np.where(mask_slice > 0))
    ymin, xmin = coords.min(axis=1)
    ymax, xmax = coords.max(axis=1)
    ymin = max(0, ymin - margin[0])
    ymax = min(img_slice.shape[0] - 1, ymax + margin[0])
    xmin = max(0, xmin - margin[1])
    xmax = min(img_slice.shape[1] - 1, xmax + margin[1])
    img_crop = img_slice[ymin:ymax+1, xmin:xmax+1]
    # Resize to crop_size
    img_crop = resize(img_crop, crop_size, order=1, mode='constant', anti_aliasing=True)
    return img_crop

# --- Preprocessing helper ---
def preprocess_slice(slice_2d):
    # Normalize and convert to uint8
    slice_2d = np.uint8(255 * np.clip(slice_2d, 0, 1))
    # DINO expects 3 channels; stack if needed
    if slice_2d.ndim == 2:
        slice_2d = np.stack([slice_2d]*3, axis=-1)
    return processor(images=slice_2d, return_tensors="pt")["pixel_values"].squeeze(0)

# --- Feature extraction ---
features, case_ids = [], []
case_dirs = sorted([d for d in os.listdir(PDGM_DIR) if d.startswith("UCSF-PDGM-")])

for case_folder in tqdm(case_dirs):
    case_path = os.path.join(PDGM_DIR, case_folder)
    num4 = case_folder.split("-")[-1].split("_")[0]
    file_case_id = f"UCSF-PDGM-{num4}"
    t1c_path = os.path.join(case_path, f"{file_case_id}_T1c_bias.nii.gz")
    mask_path = os.path.join(case_path, f"{file_case_id}_tumor_segmentation.nii.gz")
    if not (os.path.exists(t1c_path) and os.path.exists(mask_path)):
        continue

    img = nib.load(t1c_path).get_fdata().astype(np.float32)
    mask = nib.load(mask_path).get_fdata().astype(np.uint8)
    img = img / (np.max(img) + 1e-8)
    slice_embeds = []
    for i in range(img.shape[2]):
        slice_2d = img[:, :, i]
        mask_2d = mask[:, :, i]
        # Crop to tumor bbox + margin, then resize
        slice_2d_crop = crop_slice_to_bbox_with_margin(slice_2d, mask_2d, MARGIN, CROP_SIZE)
        if np.std(slice_2d_crop) < 1e-5:
            continue
        input_tensor = preprocess_slice(slice_2d_crop).to(DEVICE)
        with torch.no_grad():
            embed = vit(input_tensor.unsqueeze(0)).last_hidden_state[:, 0, :]
            proj = proj_head(embed)
            slice_embeds.append(proj.cpu().numpy().squeeze())
    if len(slice_embeds) == 0:
        continue
    case_embed = np.mean(np.stack(slice_embeds), axis=0)
    features.append(case_embed)
    case_ids.append(file_case_id)

features = np.stack(features)
np.savez(SAVE_PATH, features=features, case_ids=np.array(case_ids))
print(f"Saved DINO bbox features to {SAVE_PATH}")


/scratch/users/maxvpuyv/anaconda3/envs/brain/lib/python3.9/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 501/501 [14:52<00:00,  1.78s/it]

Saved DINO bbox features to /oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/data/features/dino_pdgm_bbox_features_1e4lr150000.npz


In [4]:

# --- Load features and metadata ---
data = np.load(SAVE_PATH, allow_pickle=True)
X = data["features"]
case_ids = data["case_ids"]
meta = pd.read_csv(METADATA_CSV).set_index("ID")

# --- Standardize case_ids to match metadata index (three digits) ---
case_ids_fixed = np.array([f"UCSF-PDGM-{int(cid.split('-')[-1]):03d}" for cid in case_ids])

# --- Filter to only cases present in metadata ---
valid_mask = np.isin(case_ids_fixed, meta.index)
X = X[valid_mask]
case_ids_fixed = case_ids_fixed[valid_mask]

# --- Prepare labels for IDH status ---
idh_status = meta.loc[case_ids_fixed, "IDH"].values
labels = np.array([0 if str(v).strip().lower() == "wildtype" else 1 for v in idh_status])

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

# --- Train regularized logistic regression classifier ---
clf = LogisticRegression(
    penalty='l2',
    max_iter=1000,
    class_weight='balanced'
)
clf.fit(X_train, y_train)

# --- Evaluate ---
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

print("Logistic Regression Classification report:")
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))


Logistic Regression Classification report:
              precision    recall  f1-score   support

           0       0.93      0.72      0.81        78
           1       0.44      0.81      0.57        21

    accuracy                           0.74        99
   macro avg       0.68      0.76      0.69        99
weighted avg       0.83      0.74      0.76        99

ROC AUC: 0.8467643467643468


In [5]:

# --- Oversample minority class in training set ---
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# --- Train Random Forest classifier ---
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train_res, y_train_res)

# --- Evaluate ---
y_pred_rf = clf_rf.predict(X_test)
y_prob_rf = clf_rf.predict_proba(X_test)[:, 1]

print("Random Forest Classification report:")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC:", roc_auc_score(y_test, y_prob_rf))


/scratch/users/maxvpuyv/anaconda3/envs/brain/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Random Forest Classification report:
              precision    recall  f1-score   support

           0       0.83      0.88      0.86        78
           1       0.44      0.33      0.38        21

    accuracy                           0.77        99
   macro avg       0.63      0.61      0.62        99
weighted avg       0.75      0.77      0.76        99

ROC AUC: 0.7087912087912088


In [6]:

# --- t-SNE Visualization ---
X_scaled = StandardScaler().fit_transform(X)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

columns = [
    "ID", "Sex", "Age at MRI", "WHO CNS Grade", "Final pathologic diagnosis (WHO 2021)",
    "MGMT status", "MGMT index", "1p/19q", "IDH", "1-dead 0-alive", "OS", "EOR",
    "Biopsy prior to imaging", "BraTS21 ID", "BraTS21 Segmentation Cohort", "BraTS21 MGMT Cohort"
]

outdir_tsne = "/oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/data/tsne_plots/PDGM_dino_bbox"
os.makedirs(outdir_tsne, exist_ok=True)

for column in columns:
    try:
        values = meta.loc[case_ids_fixed, column].values
    except KeyError:
        print(f"Column {column} not found in metadata.")
        continue

    values_str = np.array([str(v) if pd.notna(v) else "NA" for v in values])

    plt.figure(figsize=(7, 6))
    for val in np.unique(values_str):
        idx = values_str == val
        plt.scatter(X_tsne[idx, 0], X_tsne[idx, 1], label=str(val), alpha=0.7, s=20)
    plt.legend(markerscale=2, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(f"t-SNE colored by {column}")
    plt.xlabel("t-SNE-1")
    plt.ylabel("t-SNE-2")
    plt.tight_layout()
    fname = f"tsne_{column.replace(' ', '_').replace('/', '_')}.png"
    plt.savefig(os.path.join(outdir_tsne, fname))
    plt.close()
    print(f"Saved t-SNE for {column}")

print("All t-SNE plots saved")


Column ID not found in metadata.
Saved t-SNE for Sex


/tmp/ipykernel_53247/1707480134.py:32: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved t-SNE for Age at MRI
Saved t-SNE for WHO CNS Grade
Saved t-SNE for Final pathologic diagnosis (WHO 2021)
Saved t-SNE for MGMT status
Saved t-SNE for MGMT index
Saved t-SNE for 1p/19q
Saved t-SNE for IDH
Saved t-SNE for 1-dead 0-alive


/tmp/ipykernel_53247/1707480134.py:32: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved t-SNE for OS
Saved t-SNE for EOR
Saved t-SNE for Biopsy prior to imaging


/tmp/ipykernel_53247/1707480134.py:32: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved t-SNE for BraTS21 ID
Saved t-SNE for BraTS21 Segmentation Cohort
Saved t-SNE for BraTS21 MGMT Cohort
All t-SNE plots saved


In [7]:

# --- UMAP Visualization ---
reducer = umap.UMAP(random_state=42)
X_umap = reducer.fit_transform(X_scaled)

outdir_umap = "/oak/stanford/groups/ogevaert/maxvpuyv/projects/brain/data/umap_plots/PDGM_dino_bbox"
os.makedirs(outdir_umap, exist_ok=True)

for column in columns:
    try:
        values = meta.loc[case_ids_fixed, column].values
    except KeyError:
        print(f"Column {column} not found in metadata.")
        continue

    values_str = np.array([str(v) if pd.notna(v) else "NA" for v in values])

    plt.figure(figsize=(7, 6))
    for val in np.unique(values_str):
        idx = values_str == val
        plt.scatter(X_umap[idx, 0], X_umap[idx, 1], label=str(val), alpha=0.7, s=20)
    plt.legend(markerscale=2, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title(f"UMAP colored by {column}")
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.tight_layout()
    fname = f"umap_{column.replace(' ', '_').replace('/', '_')}.png"
    plt.savefig(os.path.join(outdir_umap, fname))
    plt.close()
    print(f"Saved UMAP for {column}")

print("All UMAP plots saved")

/scratch/users/maxvpuyv/anaconda3/envs/brain/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/scratch/users/maxvpuyv/anaconda3/envs/brain/lib/python3.9/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Column ID not found in metadata.
Saved UMAP for Sex


/tmp/ipykernel_53247/308477083.py:25: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved UMAP for Age at MRI
Saved UMAP for WHO CNS Grade
Saved UMAP for Final pathologic diagnosis (WHO 2021)
Saved UMAP for MGMT status
Saved UMAP for MGMT index
Saved UMAP for 1p/19q
Saved UMAP for IDH
Saved UMAP for 1-dead 0-alive


/tmp/ipykernel_53247/308477083.py:25: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved UMAP for OS
Saved UMAP for EOR
Saved UMAP for Biopsy prior to imaging


/tmp/ipykernel_53247/308477083.py:25: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


Saved UMAP for BraTS21 ID
Saved UMAP for BraTS21 Segmentation Cohort
Saved UMAP for BraTS21 MGMT Cohort
All UMAP plots saved
